In [1]:
CheckFlag = 0
epoch = 50
layer = '3+'
COCO_MODEL_PATH = './logs/heads3.h5'
TRAIN_DATASET = "dataset_aug"
VALID_DATASET = "./valid_0710"

import os
from dataset import ShapesDataset
import dataset_valid
dataset_train = ShapesDataset()
dataset_train.load_dataset(TRAIN_DATASET)
dataset_train.prepare()

dataset_valid = dataset_valid.ShapesDataset()
dataset_valid.load_dataset(VALID_DATASET)
dataset_valid.prepare()

Using TensorFlow backend.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.




173


In [2]:
"""確認用、
取り込んだデータを画像として出力する。
CheckFlagが1ならば画像が表示される。    

from mrcnn import visualize
from tqdm import tqdm

image_ids = dataset_train.image_ids
for TRAIN_DATASET in tqdm(image_ids):
    image = dataset_train.load_image(TRAIN_DATASET)
    mask, class_ids = dataset_train.load_mask(TRAIN_DATASET)
    if CheckFlag == 1 and class_ids is not None:
        visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
"""

"""確認用、
取り込んだ訓練データの配列構造が表示される。                                                                    
CheckFlagが1ならば配列構造が表示される。    

import pprint
if CheckFlag==1:
    pprint.pprint(vars(dataset_train))
    """

'確認用、\n取り込んだ訓練データの配列構造が表示される。                                                                    \nCheckFlagが1ならば配列構造が表示される。    \n\nimport pprint\nif CheckFlag==1:\n    pprint.pprint(vars(dataset_train))\n    '

In [3]:
"""
ハイパーパラメータと学習前のモデルをインポートする。 
CheckFlagが1ならばハイパーパラメータが表示される。 
"""
from mrcnn.train_config import train_Config
import mrcnn.model as modellib  

config = train_Config()
if CheckFlag==1:
    config.display()

In [4]:
"""モデルを読み込む
どの初期モデルを用いるか？
時間がかかるのと，タスクマネージャーで専用GPUが消費されていることを確認できる．

最初だとmask_rcnn_coco.h5をインストール．
"""
from mrcnn import utils

model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir="logs/model")


if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [5]:
"""GPU>1のときに用いる．
VRAMのリミッター解除
GPU_COUNT=1のときはすべてコメントアウトしてかまわない．
ここのconfigは上のハイパーパラメータとは異なるのだが，名前だけ変えてもうまく動かない汗

Args:
    config:TensorFlowのConfigであることに注意！　名前変えたいが，変えると動かぬ（直します）
"""

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()
config.allow_soft_placement=True
session = tf.Session(config=config)
KTF.set_session(session)

In [6]:
"""モデルの重みを読み込んでいるものと思われる．時間がかかる．
mask_rcnn_coco.h5学習時は  exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]を用いる
heads,3+などを学習したあとはexcludeなしのload_weightを用いる．

"""
print(COCO_MODEL_PATH)
if layer == "heads":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                    "mrcnn_bbox", "mrcnn_mask"])
else:
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
    #model.load_weights(COCO_MODEL_PATH, by_name=True)


#model.load_weights(COCO_MODEL_PATH, by_name=True)

./logs/heads3.h5


In [7]:
"""モデルの訓練を行う．本学習．時間とGPUが消費される．
Args:
    learning_rate : 学習速度
    
    epochs : エポック数
    
    layers : heads, 3+, 4+, 5+, all

"""
model.train(dataset_train, dataset_valid,
                learning_rate=0.01,
                epochs=epoch,
                layers=layer)


Starting at epoch 0. LR=0.01

Checkpoint Path: logs/model/cell_dataset20220707T1807/mask_rcnn_cell_dataset_{epoch:04d}.h5
Selecting layers to train
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b          (BatchNorm)
res3a_branch2c         (Conv2D)
res3a_branch1          (Conv2D)
bn3a_branch2c          (BatchNorm)
bn3a_branch1           (BatchNorm)
res3b_branch2a         (Conv2D)
bn3b_branch2a          (BatchNorm)
res3b_branch2b         (Conv2D)
bn3b_branch2b          (BatchNorm)
res3b_branch2c         (Conv2D)
bn3b_branch2c          (BatchNorm)
res3c_branch2a         (Conv2D)
bn3c_branch2a          (BatchNorm)
res3c_branch2b         (Conv2D)
bn3c_branch2b          (BatchNorm)
res3c_branch2c         (Conv2D)
bn3c_branch2c          (BatchNorm)
res3d_branch2a         (Conv2D)
bn3d_branch2a          (BatchNorm)
res3d_branch2b         (Conv2D)
bn3d_branch2b          (BatchNorm)
res3d_branch2c         (Conv2D)
bn3d_branch2c  

/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/50


/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

50/50 [==============================] - 7639s 153s/step - loss: 2.2241 - rpn_class_loss: 0.2437 - rpn_bbox_loss: 0.5803 - mrcnn_class_loss: 0.5051 - mrcnn_bbox_loss: 0.4517 - mrcnn_mask_loss: 0.4432 - val_loss: 5.4424 - val_rpn_class_loss: 0.2770 - val_rpn_bbox_loss: 0.8145 - val_mrcnn_class_loss: 3.5728 - val_mrcnn_bbox_loss: 0.4583 - val_mrcnn_mask_loss: 0.3198

Epoch 2/50
50/50 [==============================] - 2286s 46s/step - loss: 1.4824 - rpn_class_loss: 0.1036 - rpn_bbox_loss: 0.5795 - mrcnn_class_loss: 0.2935 - mrcnn_bbox_loss: 0.2489 - mrcnn_mask_loss: 0.2569 - val_loss: 3.1725 - val_rpn_class_loss: 1.0462 - val_rpn_bbox_loss: 1.0121 - val_mrcnn_class_loss: 0.3452 - val_mrcnn_bbox_loss: 0.3949 - val_mrcnn_mask_loss: 0.3740
Epoch 3/50
50/50 [==============================] - 2354s 47s/step - loss: 1.7068 - rpn_class_loss: 0.1512 - rpn_bbox_loss: 0.7224 - mrcnn_class_loss: 0.2691 - mrcnn_bbox_loss: 0.2809 - mrcnn_mask_loss: 0.2831 - val_loss: 3.7781 - val_rpn_class_loss: 0.44